https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/

Original DeLanda essay
https://web.archive.org/web/20240203233005/https://www.cddc.vt.edu/host/delanda/pages/algorithm.htm

In [233]:
n_bits = 20
n_pop = 5
import numpy as np
# initial population of random bitstring
pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]

In [234]:
pop

[[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0]]

In [235]:
from random import getrandbits

In [236]:
bin(getrandbits(6))

'0b110110'

In [237]:
for i in format(57, '07b'): 
    print (i)

0
1
1
1
0
0
1


# Using midi
I can use [mido](https://pypi.org/project/mido/) to send messages and program changes to the rytm

# Approach from ChatGPT

https://chatgpt.com/c/1e91a6cc-aab6-44c7-858c-47b279110f3e

In [314]:
#int.from_bytes(np.random.bytes(1),)

In [240]:
import numpy as np

def initialize_population(pop_size, num_params):
    """Initialize a population with random 7-bit binary strings."""
    return np.random.randint(0, 2, (pop_size, num_params * 7))

In [241]:
initialize_population(10, 2)

array([[0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1],
       [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1]])

In [242]:
def decode_individual(individual, num_bits_in_param=7):
    """Decode a binary individual to integer parameters."""
    num_params = len(individual) // num_bits_in_param
    return [int(''.join(map(str, individual[i*7:(i+1)*7])), 2) for i in range(num_params)]

In [243]:
decode_individual(initialize_population(10, 2)[0])

[95, 75]

In [244]:
def encode_individual(params):
    """Encode integer parameters to a binary individual."""
    return np.concatenate([np.array(list(f'{param:07b}'), dtype=int) for param in params])

In [247]:
test = encode_individual([118])

In [246]:
def fitness(individual):
    # Compute the fitness of an individual based on intensive properties.
    params = decode_individual(individual)
    frequency, timbre1, timbre2, attack, decay, sustain, noise = params
    
    # Constraints for fitness evaluation
    if not (20 <= frequency <= 2000):
        return 0  # Frequency out of desired range
    
    if not (0 <= timbre1 <= 100) or not (0 <= timbre2 <= 100):
        return 0  # Timbre parameters out of desired range
    
    if not (0 <= attack <= 40) or not (20 <= decay <= 127) or not (0 <= sustain <= 127):
        return 0  # Envelope parameters out of desired range
    
    if not (0 <= noise <= 127):
        return 0  # Noise parameter out of desired range
    
    # Example fitness calculation (maximize sum of parameters within constraints)
    return sum(params)

In [276]:
ind = initialize_population(10, 7)[0]
print(fitness(ind))
print(decode_individual(ind))

0
[110, 6, 108, 90, 2, 18, 125]


In [280]:
def select(population, fitnesses, num_mates):
    #Select individuals based on fitness for the mating pool.
    selected_indices = np.random.choice(len(population), num_mates, p=fitnesses/fitnesses.sum())
    return population[selected_indices]

In [278]:
def crossover(parent1, parent2):
    """Perform single-point crossover between two parents."""
    point = np.random.randint(1, len(parent1) - 1)
    return np.concatenate((parent1[:point], parent2[point:])), np.concatenate((parent2[:point], parent1[point:]))


In [217]:
def mutate(individual, mutation_rate):
    """Mutate an individual by flipping bits with a given mutation rate."""
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual

In [218]:
# Parameters
pop_size = 500
num_params = 7  # [frequency, timbre1, timbre2, attack, decay, sustain, noise]
num_generations = 100
mutation_rate = 0.01

In [219]:
# Initialize population
population = initialize_population(pop_size, num_params)

In [231]:
for generation in range(num_generations):
    # Compute fitnesses
    fitnesses = np.array([fitness(ind) for ind in population])
    
    # Select parents
    mating_pool = select(population, fitnesses, pop_size // 2)
    
    # Create next generation
    next_population = []
    for i in range(0, len(mating_pool), 2):
        parent1, parent2 = mating_pool[i], mating_pool[i+1]
        child1, child2 = crossover(parent1, parent2)
        next_population.extend([mutate(child1, mutation_rate), mutate(child2, mutation_rate)])
        #print(parent1)
    
    population = np.array(next_population)
    
    # Optional: print the best fitness in the current generation
    print(f"Generation {generation + 1}: Best Fitness = {max(fitnesses)}")

Generation 1: Best Fitness = 700
Generation 2: Best Fitness = 705
Generation 3: Best Fitness = 692
Generation 4: Best Fitness = 696
Generation 5: Best Fitness = 692
Generation 6: Best Fitness = 707
Generation 7: Best Fitness = 703
Generation 8: Best Fitness = 691
Generation 9: Best Fitness = 703
Generation 10: Best Fitness = 700
Generation 11: Best Fitness = 703
Generation 12: Best Fitness = 702
Generation 13: Best Fitness = 704
Generation 14: Best Fitness = 708
Generation 15: Best Fitness = 706
Generation 16: Best Fitness = 692
Generation 17: Best Fitness = 695
Generation 18: Best Fitness = 701
Generation 19: Best Fitness = 697
Generation 20: Best Fitness = 703
Generation 21: Best Fitness = 700
Generation 22: Best Fitness = 708
Generation 23: Best Fitness = 723
Generation 24: Best Fitness = 696
Generation 25: Best Fitness = 699
Generation 26: Best Fitness = 701
Generation 27: Best Fitness = 703
Generation 28: Best Fitness = 714
Generation 29: Best Fitness = 719
Generation 30: Best Fit

In [232]:
# Decode the best individual
best_individual = population[np.argmax(fitnesses)]
best_params = decode_individual(best_individual)
print(f"Best Individual: {best_individual}")
print(f"Best Parameters: {best_params}")

Best Individual: [1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1
 1 1 1 0 1 0 0 1 1 1 0 0]
Best Parameters: [127, 83, 92, 25, 124, 125, 28]
